<a href="https://colab.research.google.com/github/GajananKuchgave/EVA8_Assignments/blob/main/MNIST_2_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Neural Network to detect MNIST data and Sum of MNIST number + Random Number

#Libraries

In [14]:
#Import required libraries
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, TensorDataset
from torch.utils.data import DataLoader

import torch.nn.functional as F
import torch.optim as optim


import matplotlib.pyplot as plt
import numpy as np
import random

#Use Cuda as a device Type

In [15]:

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cuda


#Download the Training and Testing Dataset

In [16]:
# Train set
MNIST_train_set = torchvision.datasets.MNIST(
    root='./data'
    ,train=True
    ,download=True
    ,transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,)) # The mean and std have to be sequences
    ])
)

print("Training set size",len(MNIST_train_set))

MNIST_test_set = torchvision.datasets.MNIST(
    root='./data'
    ,train=False
    ,download=True
    ,transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,)) # The mean and std have to be sequences
    ])
)


Training set size 60000


#Customized dataset to read both random number and MNIST data

In [17]:
class MNISTandRandomDataset(Dataset):
  def __init__(self, MNIST_dataset):
    self.MNIST_dataset = MNIST_dataset


  def __getitem__(self, index):
    image, label = self.MNIST_dataset[index]
    randNum = random.randint(0,9)

 #Creating one hot encoding for random number 
    one_hot_randNumber = F.one_hot(torch.arange(0, 10)) 
    #add actual label and random number
    sum_label = label + randNum
    return image, label, one_hot_randNumber[randNum], sum_label

  def __len__(self):
    return len(self.MNIST_dataset)

#Training and Testing Dataset

In [18]:

train_data_set = MNISTandRandomDataset(MNIST_train_set)
test_data_set = MNISTandRandomDataset(MNIST_test_set)

In [19]:
next(iter(train_data_set))

(tensor([[[-0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
           -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
           -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
           -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
           -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
           -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
           -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
           -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
           -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
           -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242, -0.4242,
           -0.4242, -0.4242, -0.424

#Dataloader for both batch based training and Testing


In [20]:
train_loader = torch.utils.data.DataLoader(train_data_set, batch_size = 128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data_set,batch_size=128)

In [21]:
# Checking the dataset
batch = next(iter(train_loader))
image, label, random_num, sum_label = batch
print(image.shape,label,random_num, sum_label)

torch.Size([128, 1, 28, 28]) tensor([0, 3, 3, 7, 0, 8, 7, 0, 3, 2, 6, 6, 7, 0, 9, 9, 5, 5, 1, 8, 4, 1, 4, 1,
        2, 1, 6, 0, 8, 8, 4, 2, 8, 5, 5, 5, 7, 8, 9, 5, 5, 0, 0, 3, 1, 2, 1, 6,
        3, 0, 6, 5, 9, 7, 8, 0, 6, 8, 4, 3, 6, 1, 2, 3, 1, 5, 4, 8, 7, 9, 7, 7,
        2, 0, 2, 5, 1, 5, 0, 9, 8, 7, 6, 0, 4, 8, 1, 5, 1, 3, 7, 0, 8, 8, 1, 4,
        3, 2, 7, 1, 9, 2, 1, 2, 7, 8, 4, 9, 6, 0, 5, 5, 3, 1, 3, 0, 4, 8, 5, 9,
        2, 2, 9, 4, 9, 2, 6, 3]) tensor([[0, 0, 0,  ..., 0, 1, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 1, 0],
        ...,
        [0, 0, 0,  ..., 0, 1, 0],
        [1, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 1]]) tensor([ 8,  8, 11, 11,  2, 11,  9,  2, 11,  6, 12,  7, 14,  6, 16, 11, 10, 11,
         5, 12,  7,  4, 11,  9,  4,  6, 12,  8, 14, 10,  7, 11,  8,  8,  6, 11,
        14, 16, 14,  7,  9,  8,  9,  9,  1,  7,  4, 11,  9,  6, 13,  6, 11, 14,
        15,  5, 10,  9,  9,  6,  9, 10,  8, 10,  1,  5, 12, 10,  9,  9, 14,  7,


#Network Class for MNIST and Random NN Model

In [22]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) 
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) 
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 64, 3, padding=1)
        self.conv4 = nn.Conv2d(64, 128, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(128, 128, 3)
        self.conv6 = nn.Conv2d(128, 256, 3)
        self.conv7 = nn.Conv2d(256, 10, 3)

        # Linear fully connected layer 1 and 2 used for mixing both MNIST number and Sum with Random Number
        self.fc1 = nn.Linear(20, 64)    
        # Total of 19 outputs for MNIST + Random output layer (0 - 18)           
        self.fc2 = nn.Linear(64, 19) 

    def forward(self, x, rand_num):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = F.relu(self.conv7(x))
        #x = x.view(-1, 10*3*3)
        x = torch.flatten(x,1)
  
        new_x = torch.cat((x, rand_num), dim=1) # Concatenated both MNIST image data with Random number

        new_x = self.fc2(F.relu(self.fc1(new_x))) 

        return F.log_softmax(x, dim = 1), F.log_softmax(new_x, dim = 1)

#Taking Model from Network

In [23]:
model = Network()

#Training and Testing of a Model

In [24]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    epoch_loss = 0
    for batch_idx, (image, label, randNum, sum_label) in enumerate(pbar):
        image, label, randNum,sum_label = image.to(device), label.to(device),randNum.to(device),sum_label.to(device)
        optimizer.zero_grad()
        output, sum_output = model(image,randNum )
        mnist_loss = F.cross_entropy(output, label)
        sum_loss = F.cross_entropy(sum_output,sum_label)
        loss = (mnist_loss + sum_loss)/2
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'Training_loss={loss.item():.3f} batch_id={batch_idx}')
    train_loss = epoch_loss / len(train_loader)
    return train_loss

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    sum_correct = 0
    with torch.no_grad():
        for image, label, randNum, sum_label in test_loader:
            image, label, randNum,sum_label = image.to(device), label.to(device),randNum.to(device),sum_label.to(device)
            output, sum_output = model(image, randNum)
            #test_loss += F.nll_loss(output, label, reduction='sum').item()  # sum up batch loss
            test_loss += (F.cross_entropy(output, label) + F.cross_entropy(sum_output, sum_label))/2
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            sum_pred = sum_output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(label.view_as(pred)).sum().item()
            sum_correct += pred.eq(sum_label.view_as(sum_pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    MNIST_Accuracy = 100. * correct/len(test_loader.dataset)
    Addtion_Accuracy = 100. * sum_correct/len(test_loader.dataset)
   
    print(f' set: Average loss: {test_loss:.3f},   MNIST Accuracy:{MNIST_Accuracy:.3f},   Addition_Accuracy:{Addtion_Accuracy:.3f}')
    print('\n')
    return test_loss

In [25]:
model = Network().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.9)

train_loss_values = []
test_loss_values = []

for epoch in range(1, 11):
    train_loss = train(model, device, train_loader, optimizer, epoch)
    test_loss = test(model, device, test_loader)

    train_loss_values.append(train_loss)
    test_loss_values.append(test_loss)

Training_loss=1.448 batch_id=468: 100%|██████████| 469/469 [00:25<00:00, 18.53it/s]


 set: Average loss: 0.012,   MNIST Accuracy:91.220,   Addition_Accuracy:9.920




Training_loss=1.245 batch_id=468: 100%|██████████| 469/469 [00:25<00:00, 18.28it/s]


 set: Average loss: 0.010,   MNIST Accuracy:96.360,   Addition_Accuracy:10.010




Training_loss=1.138 batch_id=468: 100%|██████████| 469/469 [00:25<00:00, 18.42it/s]


...

 set: Average loss: 0.005,   MNIST Accuracy:98.980,   Addition_Accuracy:9.950




Training_loss=0.417 batch_id=468: 100%|██████████| 469/469 [00:25<00:00, 18.10it/s]


 set: Average loss: 0.004,   MNIST Accuracy:98.780,   Addition_Accuracy:9.590




Training_loss=0.325 batch_id=468: 100%|██████████| 469/469 [00:25<00:00, 18.43it/s]


 set: Average loss: 0.003,   MNIST Accuracy:98.800,   Addition_Accuracy:10.430


